<a href="https://colab.research.google.com/github/asigalov61/Quintessential-Viterbi/blob/main/Quintessential_Viterbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quintessential Viterbi (v. 0.9)

***

### Full-featured Viterbi MIDI Music Augmentator

***

#### Project Los Angeles

#### Tegridy Code 2020

In [ ]:
#@title Install dependencies
print('Installing dependencies...')
print('Please stand-by...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

print('Installing FluidSynth...')
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Done! Enjoy! :)')

In [ ]:
#@title Import Modules
import constants
import chords_lib, chord_inference

import midi_io, midi_synth
import sequences_lib
import melodies_lib, melody_inference
import notebook_utils

import copy

from google.colab import files
import IPython

In [ ]:
#@title Generate Agumented Viterbi Music from your MIDI
full_path_to_MIDI_file = "/content/Sample_MIDI.mid" #@param {type:"string"}

melody_ns = midi_io.midi_file_to_note_sequence(full_path_to_MIDI_file)

mel_q = sequences_lib.quantize_note_sequence(melody_ns, steps_per_quarter=32)

mel0 = melodies_lib.Melody()
mel0.from_quantized_sequence(mel_q,
                              search_start_step=0,
                              instrument=0,
                              gap_bars=1,
                              ignore_polyphonic_notes=True,
                              pad_end=False,
                              filter_drums=True)

mel1 = mel0.to_sequence(velocity=100, 
                instrument=0, 
                program=0, 
                sequence_start_time=0.0, 
                #qpm=120.0
                )

mel = sequences_lib.remove_redundant_data(mel1)                                                  

chord_inference.infer_chords_for_sequence(mel_q,
                              #chords_per_bar=8,
                              key_change_prob=0.9,
                              chord_change_prob=0.9,
                              chord_pitch_out_of_key_prob=0.9,
                              chord_note_concentration=5,
                              add_key_signatures=True
                              )

chords = [(ta.text, ta.time) for ta in mel_q.text_annotations]

chord = [lis[0] for lis in chords]
chord_times = [lis[1] for lis in chords]
chords_lib.add_chords_to_sequence(mel, chord, chord_times)
chords_lib.BasicChordRenderer(velocity=80, 
                                  instrument=1, 
                                  program=1, 
                                  octave=4, 
                                  bass_octave=3).render(mel)

augmented_sequence = sequences_lib.augment_note_sequence(
        mel,
        min_stretch_factor=-0.5,
        max_stretch_factor=2,
        min_transpose=-15,
        max_transpose=15,
        min_allowed_pitch=30,
        max_allowed_pitch=90,
        delete_out_of_range_notes=True)

mel = sequences_lib.remove_redundant_data(augmented_sequence)

print('Original MIDI composition:')
notebook_utils.plot_sequence(melody_ns)
print('Augmented Viterbi MIDI composition:')
notebook_utils.plot_sequence(mel)
print('Some stats:')
print('Note Histogram:', melodies_lib.Melody.get_note_histogram(mel0))
print('Major Key Histogram:', melodies_lib.Melody.get_major_key_histogram(mel0))
print('Major Key:', melodies_lib.Melody.get_major_key(mel0))
print('Synthesizing the output Viterbi MIDI. Please stand-by... ')
synth=midi_synth.fluidsynth(mel, sample_rate=32000, sf2_path='/usr/share/sounds/sf2/FluidR3_GM.sf2')
notebook_utils.colab_play(synth, sample_rate=32000, autoplay=True)
print('Downloading Viterbi MIDI composition... ')
print('Task complete! Enjoy! :)')

#midi_io.sequence_proto_to_midi_file(melody_ns, 'Original_Melody.mid')
midi_io.sequence_proto_to_midi_file(mel, 'Viterbi_Melody.mid')

#files.download('Original_Melody.mid')
files.download('Viterbi_Melody.mid')